<a href="https://colab.research.google.com/github/physicsIS/Physics-in-Arts/blob/main/Phis_gamma_ray_markov_chain_poems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generative Poetry

In this notebook, we adapt the algorithm proposed in *Astrophysical Narratives: Poetic Representations of Gamma-Ray Emission via Markov Chains*. This is an algorithm for text generation that adapts the classic Markov chain model to a small corpus, using the distribution of astrophysical gamma-ray sources detected by the FermiLAT satellite as the state matrix.

A long time ago, I stopped caring whether the world knows my mind, my soul, and my heart. So here, in the corpus (in the folder), are the original writings from my hospitalization and the corresponding letters. Please be discreet; what you read is the work of a broken soul that has tried to leave this world but has only managed to give a voice to the stars. These are the visible fragments of a psychiatric patient’s mind, the tears spilled onto paper.

Additionally, there are included writings and songs I’ve cherished throughout my life. Alejandra Pizarnik, Love of Lesbian, Carlos Sadness, among others, accompany my writings.

Keep in mind that most of the results will not make sense, run the poem generator several times until you find a verse that fits to you.
---



In [ ]:
# Install necessary dependencies
!pip install opencv-python-headless numpy language_tool_python spacy nltk
!python -m spacy download es_dep_news_trf  #------------------------------------ Download the model 'es_dep_news_trf'
!python -m spacy download es_core_news_sm  #------------------------------------ Download the model 'es_core_news_sm'
!pip install gpt4all
!pip install translate
!pip install deep-translator

import cv2  #------------------------------------------------------------------- OpenCV for image processing
from google.colab.patches import cv2_imshow  #---------------------------------- Display images in Google Colab
import numpy as np  #----------------------------------------------------------- NumPy for numerical operations
import random  #---------------------------------------------------------------- Random number generation
import language_tool_python  #-------------------------------------------------- Grammar checking tool
import spacy  #----------------------------------------------------------------- Natural language processing
import nltk  #------------------------------------------------------------------ NLTK for natural language processing
from nltk.corpus import stopwords, wordnet  #----------------------------------- Stopwords and WordNet for natural language processing
import re  #-------------------------------------------------------------------- Regular expressions for text processing
from gpt4all import GPT4All  #-------------------------------------------------- GPT-4 All language model for text correction
from translate import Translator  #--------------------------------------------- Package for text translation
from deep_translator import GoogleTranslator  #--------------------------------- Package for text translation
from IPython.display import HTML  #--------------------------------------------- Display HTML content in Colab
import math  #------------------------------------------------------------------ Math operations


In [ ]:
# Load the GPT-4 All model
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")
# If the model is not already downloaded locally (or in Drive), set "allow_download=True" to download it.
# You can choose the model according to your preference from GPT4ALL or Hugging Face, just download the .gguf file

# Load the spaCy model for text analysis
nlp = spacy.load("es_dep_news_trf")  #------------------------------------------ Using the transformer model specific to Spanish
nlp_sm = spacy.load("es_core_news_sm")  #------------------------------------------ Using the small model specific to Spanish
nlp.max_length = 2000000  #----------------------------------------------------- Increase the processing limit to 2 million characters (adjust as necessary)
nlp_sm.max_length = 2000000  #----------------------------------------------------- Increase the processing limit to 2 million characters (adjust as necessary)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
# Tokenize and lemmatize the corpus of poems using spaCy
def load_poem(path):  #--------------------------------------------------------- Function for tokenization and lemmatization while preserving punctuation
    """"
    The tokenization function uses transformers,
    which can slow down the code depending on
    the size of the corpus being loaded."
    """
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    doc = nlp(text)  #---------------------------------------------------------- Process the text with spaCy
    # Process tokens while preserving punctuation and only removing spaces
    processed = [
        token.text if token.is_punct else token.lemma_  #----------------------- Keep original text if punctuation
        for token in doc
        if not token.is_space  #------------------------------------------------ Only removes spaces
    ]
    return processed

# Only tokenize the corpus
def load_tokenized_poem(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    doc = nlp_sm(text)  #---------------------------------------------------------- Process the text with spaCy to obtain tokens
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    return tokens

# Function to get synonyms [Spanish] using WordNet
def get_synonym_es(word):
    try:
        # Translate the word to English
        translator_es_en = GoogleTranslator(source='es', target='en')
        word_en = translator_es_en.translate(word).lower()

        # Get synonyms in English
        synonyms = set()
        for synset in wordnet.synsets(word_en):
            for lemma in synset.lemmas():
                # Avoid adding the same word as a synonym
                if lemma.name().lower() != word_en:
                    synonyms.add(lemma.name())

        # If no synonyms are found, return the original word
        if not synonyms:
            print(f"No synonyms found for '{word}'")
            return word

        # Select a random synonym
        synonym_en = random.choice(list(synonyms))

        # Replace underscores with spaces
        synonym_en = synonym_en.replace('_', ' ')

        # Translate the synonym back to Spanish
        translator_en_es = GoogleTranslator(source='en', target='es')
        synonym_es = translator_en_es.translate(synonym_en)
        print(synonyms)
        return synonym_es

    except Exception as e:
        print(f"Error processing the word: {str(e)}")
        return word

# Function to get synonyms [English] using WordNet
# def get_synonym_en(word):
#     try:
#         # Get synonyms in English
#         synonyms = set()
#         for synset in wordnet.synsets(word_en):
#             for lemma in synset.lemmas():
#                 # Avoid adding the same word as a synonym
#                 if lemma.name().lower() != word_en:
#                     synonyms.add(lemma.name())

#         # If no synonyms are found, return the original word
#         if not synonyms:
#             print(f"No synonyms found for '{word}'")
#             return word

#         # Select a random synonym
#         synonym_en = random.choice(list(synonyms))

#         # Replace underscores with spaces
#         synonym_en = synonym_en.replace('_', ' ')

#         # Translate the synonym back to Spanish
#         print(synonyms)
#         return synonym_en

#     except Exception as e:
#         print(f"Error processing the word: {str(e)}")
#         return word


# Function to correct grammar using LanguageTool
def grammar_corrector(text):
    tool = language_tool_python.LanguageTool('es')
    return tool.correct(text)

# Function to correct the poem using a language model via GPT-4 All
def correct_poem(poem): #------------------------------------------------------- Change the promtp to taste depending on the language
    correction_prompt = f"Corrige el siguiente poema (verso) en español, conjugando adecuadamente las palabras lematizadas para darles un sentido coherente y poético, sin alejarte del significado original. Organiza el resultado de manera que fluya como un poema natural, con la estructura y la puntuación adecuada. Devuelve solo el texto corregido dentro de corchetes, sin incluir nada más:\n\n{poem}"

    with model.chat_session() as chat:  #--------------------------------------- Initialize the model in chat mode for better response
        corrected_poem = chat.generate(correction_prompt)
    return corrected_poem.strip()


In [ ]:
#Data retrieval from GitHub. The corpus consists of the author's writings and the ROI described in the article.
!wget https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Corpus/corpus_lemmatized.txt
!wget https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Corpus/corpus_tokenized.txt

!wget -O ra7200_dec99799.png https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Fuentes/ra7200_dec99799.png
!wget -O ra7200_dec99799_2.png https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Fuentes/ra7200_dec99799_2.png
!wget -O ra7200_dec99799_3.png https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Fuentes/ra7200_dec99799_3.png
!wget -O ra7200_dec99799_4.png https://raw.githubusercontent.com/physicsIS/Physics-in-Arts/refs/heads/main/Poetry/Fuentes/ra7200_dec99799_4.png

In [ ]:
"""
In this section, the corpus is loaded and processed.
This can be any plain text file. The longer the corpus,
the better the results the algorithm will produce.
Users are encouraged to utilize not only their own poems
or texts but also poems, books, and songs they enjoy.
Humans learn by imitation, especially in artistic matters.
We are all influenced by external factors, references,
and more.
"""
# Load image from a specified path
image_0 = cv2.imread("/content/ra7200_dec99799.png", cv2.IMREAD_GRAYSCALE)
image_1 = cv2.imread("/content/ra7200_dec99799_2.png", cv2.IMREAD_GRAYSCALE)
image_2 = cv2.imread("/content/ra7200_dec99799_3.png", cv2.IMREAD_GRAYSCALE)
image_3 = cv2.imread("/content/ra7200_dec99799_4.png", cv2.IMREAD_GRAYSCALE)
cv2_imshow(image_0)  #------------------------------------------------------------ Display the image in a window
cv2_imshow(image_1)  #------------------------------------------------------------ Display the image in a window
cv2_imshow(image_2)  #------------------------------------------------------------ Display the image in a window
cv2_imshow(image_3)  #------------------------------------------------------------ Display the image in a window
print("Image loaded.")

"""
# Load text lemmatized files for poems from a specified path (CAUTION THIS MAY SLOW DOWN THE CODE DEPENDING ON SIZE OF THE TEXT)
poems = load_poem("PATH/TO/YOUR/TEXT.txt")
# etc. add as many files as you need.
"""
# Load text tokenized files for poems from a specified path
poems_tokenized = load_tokenized_poem("/content/Corpus_lemmatized.txt")
#same here, add as many files as you need them :)

print("Poem files loaded.")


# Combine all lemmatized poems into a single list
# Here you can select if the corpus is lemmatized or tokenized by changing the string "poems"
#poems = poems
poems = poems_tokenized
print("Poems loaded.")

"""
# Save the result of tokenization and lemmatization to a file with one word per line
#This is useful for compare the correct function of the tokenization and lemmatization funtions
with open("lemmatized_poems.txt", "w", encoding="utf-8") as file:
    file.write('\n'.join(poems))  #--------------------------------------------- Each word on a new line

with open("Corpus.txt", "w", encoding="utf-8") as file:
    file.write(' '.join(poems))
"""

In [ ]:
# Create a Markov chain dictionary avoiding duplicates but maintaining order
chain = {}
for index, word in enumerate(poems[1:], 1):
    key = poems[index - 1]
    if key not in chain:
        chain[key] = []
    if word not in chain[key]:
        chain[key].append(word)

print("Markov chain dictionary created.")


In [ ]:
# Generate a poem
word1 = random.choice(list(chain.keys()))
message = word1.capitalize()
count = 7  #------------------------------------------------------------------- Total number of words in the poem
words_per_line = 10  #---------------------------------------------------------- Number of words per line

while len(message.split(' ')) < count:

    """This step is necessary to avoid stationary states;
    it is the same gamma-ray distribution with different
    color scale treatments and inversions."""
    selc = random.randint(0, 3)
    print()
    if selc == 0:
        print("imagen usada: 1")
        image = image_0
    elif selc == 1:
        print("imagen usada: 2")
        image = image_1
    elif selc == 2:
        print("imagen usada: 3")
        image = image_2
    elif selc == 3:
        print("imagen usada: 4")
        image = image_3


    wp = np.ones(len(chain[word1])) / len(chain[word1])
    n = len(wp)

    # Create Markov matrix from the image
    if len(wp) <= image.shape[0]:
        markov_n = np.zeros((n, n))
        num_sub = image.shape[0] // n
        for i in range(n):
            for j in range(n):
                sub_mat = image[i*num_sub:(i+1)*num_sub, j*num_sub:(j+1)*num_sub]
                markov_n[i, j] = np.mean(sub_mat)
    else:
        markov_n = cv2.resize(image, (n, n))

    # Normalize rows
    markov_n = markov_n.astype(np.float64)
    markov_n /= markov_n.sum(axis=1, keepdims=True)

    # Choose the next word based on the matrix
    next_w = np.dot(wp, markov_n)
    if len(next_w) > 1:
        selc_w = np.argmax(next_w)
        word2 = chain[word1][selc_w]
    else:
        word2 = random.choice(list(chain.keys()))


    print(next_w[np.argmax(next_w)])
    print("Current word: " + word1)
    print("Number of options: " + str(len(chain[word1])))
    if len(chain[word1]) <= 1:
        print("No options")
    print(chain[word1])
    print("Next word: " + word2)

    # Get synonyms if needed
    if np.amax(next_w) > 0.8:
        word2_2 = get_synonym_es(word2)
        #word2_2 = get_synonym_en(word2) #-------------------------------------- If you are using a corpus in English, use this function.
                                                                                #Uncomment it in the function declaration section.
        #print("Condition: yes")
        print("Synonym used: " + word2_2)
    else:
        word2_2 = word2
        #print("Condition: no")

    word1 = word2
    message += ' ' + word2_2

    # Add a line break after a certain number of words
    if len(message.split(' ')) % words_per_line == 0:
        message = message.rstrip() + '\n'

print("Generated poem:")
print(message)


In [ ]:
# Apply final grammar correction
corrected_message = grammar_corrector(message)
print("First correction: "+corrected_message)

# Correct the poem using GPT-4 All (it may or may not works well depending on the capabilities of the model to follow instructions)
output_data = correct_poem(corrected_message)

final_message = f"{message}\n\nFinalorrected version:\n{output_data}"

print(final_message)

print("Process completed.")




---


### Application of Concrete Poetry

**[Optional: The previous result is a fully functional poem, which is the objective of the algorithm. From this point onward, only visual formatting will be applied.]**

Depending on the energy value of the last word of the poem, a corresponding function will be selected to create a specific visual shape.



In [ ]:
# Convert text to HTML with random words in italics
def text_to_html_random_italics(text, num_italics=1):
    words = text.split()
    num_italics = min(num_italics, len(words))
    random_indices = random.sample(range(len(words)), num_italics)
    for idx in random_indices:
        words[idx] = f"<i>{words[idx]}</i>"
    return ' '.join(words)

# Shows the text formatted in Colab
def display_formatted_text(text):
    display(HTML(text))

# 1. Diamond
def create_diamond(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    max_width = len(words_list) // 2 + 1
    html_output = []

    for i in range(1, max_width + 1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))
    for i in range(max_width - 1, 0, -1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 2. Triangle
def create_triangle(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i in range(1, len(words_list) + 1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 3. Inverted triangle
def create_reverse_triangle(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i in range(len(words_list), 0, -1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 4. Hourglass
def create_hourglass(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i in range(len(words_list), 0, -1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))
    for i in range(2, len(words_list) + 1):
        line = ' '.join(words_list[:i])
        html_output.append(line.center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 5. Spiral
def create_spiral(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    size = math.ceil(math.sqrt(len(words_list)))
    matrix = [['' for _ in range(size)] for _ in range(size)]

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    x, y = 0, 0
    dir_idx = 0
    word_idx = 0

    while word_idx < len(words_list):
        if (0 <= x < size and 0 <= y < size and matrix[x][y] == ''):
            matrix[x][y] = words_list[word_idx]
            word_idx += 1

        next_x = x + directions[dir_idx][0]
        next_y = y + directions[dir_idx][1]

        if (next_x < 0 or next_x >= size or next_y < 0 or next_y >= size or matrix[next_x][next_y] != ''):
            dir_idx = (dir_idx + 1) % 4

        x += directions[dir_idx][0]
        y += directions[dir_idx][1]

    html_output = []
    for row in matrix:
        line = ' '.join(word for word in row if word)
        if line.strip():
            html_output.append(line)

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 6. Zigzag
def create_zigzag(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i, word in enumerate(words_list):
        spaces = '&nbsp;' * (4 * (i % 6))
        html_output.append(f'{spaces}{word}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 7. Circle aprox
def create_circle(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    radius = len(words_list) // 4
    for i, word in enumerate(words_list):
        angle = (i * 2 * math.pi) / len(words_list)
        spaces = '&nbsp;' * int(radius + radius * math.cos(angle))
        html_output.append(f'{spaces}{word}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 8. Wave
def create_wave(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i, word in enumerate(words_list):
        spaces = '&nbsp;' * (10 + int(5 * math.sin(i * 0.5)))
        html_output.append(f'{spaces}{word}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 9. Cross
def create_cross(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    middle = len(words_list) // 2
    html_output = []

    for i in range(len(words_list)):
        if i == middle:
            html_output.append(' '.join(words_list))
        else:
            spaces = '&nbsp;' * (len(words_list[middle]) * 2)
            html_output.append(f'{spaces}{words_list[i]}')

    final_html = '<div style="font-family: monospace; text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 10. Cascade
def create_cascade(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for i, word in enumerate(words_list):
        spaces = '&nbsp;' * (i * 4)
        html_output.append(f'{spaces}{word}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 11. Square
def create_square(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    size = math.ceil(math.sqrt(len(words_list)))
    html_output = []

    for i in range(size):
        line_words = words_list[i*size:min((i+1)*size, len(words_list))]
        line = ' '.join(line_words)
        html_output.append(line.center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 12. Random Scatter
def create_random_scatter(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []

    for word in words_list:
        spaces = '&nbsp;' * random.randint(0, 40)
        html_output.append(f'{spaces}{word}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 13. Arrow
def create_arrow(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    middle = len(words_list) // 2
    html_output = []

    # Punta de la flecha
    for i in range(middle):
        line = ' '.join(words_list[i:i+1])
        html_output.append(line.center(50))

    # Línea central
    html_output.append(' '.join(words_list[middle:]).center(50))

    final_html = '<div style="text-align: center;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 14. Pyramid
def create_pyramid(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []
    step_size = 2

    for i in range(0, len(words_list), step_size):
        line = ' '.join(words_list[i:i+step_size])
        spaces = '&nbsp;' * (i * 2)
        html_output.append(f'{spaces}{line}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

# 15. Helix
def create_helix(words):
    formatted_words = text_to_html_random_italics(words)
    words_list = formatted_words.split()
    html_output = []
    amplitude = 20

    for i, word in enumerate(words_list):
        spaces_left = '&nbsp;' * int(amplitude + amplitude * math.sin(i * 0.5))
        spaces_right = '&nbsp;' * int(amplitude + amplitude * math.cos(i * 0.5))
        html_output.append(f'{spaces_left}{word}{spaces_right}')

    final_html = '<div style="font-family: monospace;">' + '<br>'.join(html_output) + '</div>'
    display_formatted_text(final_html)

def create_concrete_poetry(text, form, max_value = 15):
#form (float or str): Decimal number (0 to max_value) or name of the desired form
#max_value (float): Maximum value for the input range (default: 15.0)
    available_functions = [
        create_diamond,
        create_triangle,
        create_reverse_triangle,
        create_hourglass,
        create_spiral,
        create_zigzag,
        create_circle,
        create_wave,
        create_cross,
        create_cascade,
        create_square,
        create_random_scatter,
        create_arrow,
        create_pyramid,
        create_helix
    ]

    #Dictionary for shape names
    named_functions = {
        'diamond': create_diamond,
        'triangle': create_triangle,
        'reverse_triangle': create_reverse_triangle,
        'hourglass': create_hourglass,
        'spiral': create_spiral,
        'zigzag': create_zigzag,
        'circle': create_circle,
        'wave': create_wave,
        'cross': create_cross,
        'cascade': create_cascade,
        'square': create_square,
        'random_scatter': create_random_scatter,
        'arrow': create_arrow,
        'pyramid': create_pyramid,
        'helix': create_helix
    }

    if isinstance(form, (int, float)):
        if form < 0 or form > max_value:
            form = random.random() * max_value
            print(f"Value out of range. Using random value: {form:.3f}")
        # Map the decimal value to the function index
        normalized_value = form / max_value  # Normalize the value between 0 and 1
        index = int(normalized_value * len(available_functions))
        # Adjust for the case where form == max_value
        if index == len(available_functions):
            index = len(available_functions) - 1

        selected_function = available_functions[index]
        print(f"Value {form:.3f} mapped to shape #{index + 1}")
    elif isinstance(form, str):
        if form.lower() not in named_functions:
            form = random.random() * max_value
            print(f"Invalid shape name. Using random value: {form:.3f}")
            normalized_value = form / max_value
            index = int(normalized_value * len(available_functions))
            selected_function = available_functions[index]
        else:
            selected_function = named_functions[form.lower()]
    else:
        raise ValueError("The 'form' parameter must be a number or a string")

    selected_function(text)


In [ ]:
num = ((np.argmax(next_w)-np.amin(markov_n))*15)/(np.argmax(markov_n) - np.amin(markov_n))
print(num)
create_concrete_poetry(output_data, form=num, max_value=15)